In [1]:
import pandas as pd
import urllib.request
import json
import sqlite3

In [3]:
#read data from today
df = pd.read_json("/workspaces/via_rail/raw_data/Via_data_2025-03-15 22:17:39.545499.json")
#transpose data
df_t = df.transpose()
#shift train name into column
df_t = df_t.reset_index()

# Pull Daily Data from JSON

In [4]:
via_day_data = df_t[['times','index','departed','arrived', 'from', 'to', ]]

# Initialize an empty list to store DataFrames with IDs
flattened_dfs = []

In [5]:
# Iterate over each row in x
for idx, row in via_day_data.iterrows():

    # Iterate over each DataFrame in the sublist
    for df in row['times']:
        # Add the ID as a column to the DataFrame
        df['ID'] = row['index']
        df['departed'] = row['departed']
        df['arrived'] = row['arrived']
        df['from'] = row['from']
        df['to'] = row['to']
        # Append the DataFrame to the list
        flattened_dfs.append(df)

test_2 = pd.DataFrame(flattened_dfs)

#Clean up datetimes
test_2['estimated'] = pd.to_datetime(test_2['estimated'], errors = 'coerce')
test_2['scheduled'] = pd.to_datetime(test_2['scheduled'], errors = 'coerce')

In [6]:
via_day_data_clean = test_2[['station','code', 'ID','scheduled','estimated','departed','diffMin', 'arrived','from','to']]

In [7]:
con = sqlite3.connect("via_data.db")
cur = con.cursor()

# Ensure all locations are present

In [ ]:
sqlite_insert_with_param = """INSERT INTO tbl_location_v2(LOCATION, LOCATION_CODE) VALUES (?, ?) ON CONFLICT(LOCATION) DO NOTHING;"""
# Iterate over each row in x
for idx, row in via_day_data_clean.iterrows():
    #change to upper case to align with to/from variables
    location_i = row['station'].upper()
    location_code_i = row['code']
    data_tuple = (location_i, location_code_i)

    print(data_tuple)

    return_v = cur.execute(sqlite_insert_with_param, data_tuple)
    print(return_v)

con.commit()
#cur.close()  

# Ensure all routes are present

In [23]:
sqlite_insert_with_param = """INSERT INTO tbl_route_v3(ROUTE_START, ROUTE_END) VALUES (?, ?) ON CONFLICT(ROUTE_START, ROUTE_END) DO NOTHING;"""

query_location = """SELECT LOCATION_ID FROM tbl_location_v2 WHERE LOCATION = ?"""
# Iterate over each row in x
for idx, row in via_day_data_clean.iterrows():
    #change to upper case to align with to/from variables
    from_i = row['from']
    to_i = row['to']

    #get to index
    cur.execute(query_location, (to_i,))
    row = cur.fetchone()
    to_idx = row[0]

    #get from index
    cur.execute(query_location, (from_i,))
    row = cur.fetchone()
    from_idx = row[0]

    #add to route table
    data_tuple = (from_idx, to_idx)
    return_v = cur.execute(sqlite_insert_with_param, data_tuple)

con.commit()
#cur.close()  

# Ensure all Trains are present

In [29]:
sqlite_insert_with_param = """INSERT INTO tbl_train_v3(TRAIN_NUMBER, ROUTE) VALUES (?, ?) ON CONFLICT(TRAIN_NUMBER, ROUTE) DO NOTHING;"""

query_location = """SELECT LOCATION_ID FROM tbl_location_v2 WHERE LOCATION = ?"""
query_route = """SELECT ROUTE_ID FROM tbl_route_v3 WHERE ROUTE_START = ? AND ROUTE_END = ?"""
# Iterate over each row in x
for idx, row in via_day_data_clean.iterrows():
    #change to upper case to align with to/from variables
    from_i = row['from']
    to_i = row['to']
    train_num = row['ID']

    #get to index
    cur.execute(query_location, (to_i,))
    row = cur.fetchone()
    to_idx = row[0]

    #get from index
    cur.execute(query_location, (from_i,))
    row = cur.fetchone()
    from_idx = row[0]

    #get route
    cur.execute(query_route, (from_idx,to_idx))
    row = cur.fetchone()
    route_idx = row[0]

    #add to route table
    data_tuple = (train_num, route_idx)
    return_v = cur.execute(sqlite_insert_with_param, data_tuple)

con.commit()
#cur.close() 

# Ensure all stops are present

In [36]:
sqlite_insert_with_param = """INSERT INTO tbl_stop_v4(TRAIN, ROUTE_STOP, ROUTE_NEXT_STOP) VALUES (?, ?, ?) ON CONFLICT(TRAIN, ROUTE_STOP, ROUTE_NEXT_STOP) DO NOTHING;"""

query_location = """SELECT LOCATION_ID FROM tbl_location_v2 WHERE LOCATION = ?"""
query_route = """SELECT ROUTE_ID FROM tbl_route_v3 WHERE ROUTE_START = ? AND ROUTE_END = ?"""
query_train = """SELECT TRAIN_ID FROM tbl_train_v3 WHERE TRAIN_NUMBER = ? AND ROUTE = ?"""

#define previous row/stop data
prev_stop_location_id = None
train_id = None

# Iterate over each row in x
for idx, row in via_day_data_clean.iterrows():
    #change to upper case to align with to/from variables
    from_i = row['from']
    to_i = row['to']
    train_num = row['ID']
    location_i = row['station'].upper()
    #location_code_i = row['code']

    #get to index
    cur.execute(query_location, (location_i,))
    row = cur.fetchone()
    location_idx = row[0]

    #get to index
    cur.execute(query_location, (to_i,))
    row = cur.fetchone()
    to_idx = row[0]

    #get from index
    cur.execute(query_location, (from_i,))
    row = cur.fetchone()
    from_idx = row[0]

    #get route
    cur.execute(query_route, (from_idx, to_idx))
    row = cur.fetchone()
    route_idx = row[0]

    #get train
    cur.execute(query_train, (train_num, route_idx))
    row = cur.fetchone()
    train_idx = row[0]

    #add to stop table

    #if no previous stop data (i.e. this is the first row for the train)
    #nothing to write to database just save indexes
    if prev_stop_location_id is None:
        prev_stop_location_id = location_idx
        train_id = train_idx
    #if train id does not equal train_idx then we are on a new train (first stop)
    #nothing to write to database just save indexes
    elif train_id != train_idx:
        prev_stop_location_id = location_idx
        train_id = train_idx
    else:
        data_tuple = (train_idx, prev_stop_location_id, location_idx)
        return_v = cur.execute(sqlite_insert_with_param, data_tuple)

        #check if location is equal to end of route
        #write last stop row to database
        if location_idx == to_idx:
            data_tuple = (train_idx, location_idx, location_idx)
            return_v = cur.execute(sqlite_insert_with_param, data_tuple)

        prev_stop_location_id = location_idx


con.commit()
#cur.close()

# Add daily data to table

In [ ]:
via_day_data_clean['next_station'] = via_day_data_clean.groupby('ID')['station'].shift(-1)
#last row of each train has NA - try to impute
#via_day_data_clean.loc[(via_day_data_clean.station.upper() == via_day_data_clean.to), "next_stop"] = via_day_data_clean.to

#create depature time column
via_day_data_clean['train_departure_schedule'] = via_day_data_clean.groupby('ID')['scheduled'].transform('min')

In [ ]:
sqlite_insert_with_param = """INSERT INTO tbl_via_data_v4(TRAIN_STOP, SCHEDULE_DATETIME, ARRIVAL_DATETIME, MINUTES_LATE, DATE_TRAIN) VALUES (?, ?, ?, ?, ?) ON CONFLICT(TRAIN_STOP, DATE_TRAIN) DO NOTHING;"""

query_location = """SELECT LOCATION_ID FROM tbl_location_v2 WHERE LOCATION = ?"""
query_route = """SELECT ROUTE_ID FROM tbl_route_v3 WHERE ROUTE_START = ? AND ROUTE_END = ?"""
query_train = """SELECT TRAIN_ID FROM tbl_train_v3 WHERE TRAIN_NUMBER = ? AND ROUTE = ?"""
query_stop = """SELECT STOP_ID FROM tbl_stop_v4 WHERE TRAIN = ? AND ROUTE_STOP = ? AND ROUTE_NEXT_STOP = ?"""

# Iterate over each row in x
for idx, row in via_day_data_clean.iterrows():
    #change to upper case to align with to/from variables
    from_i = row['from']
    to_i = row['to']
    train_num = row['ID']
    location_i = row['station'].upper()
    next_location_i = row['next_station']

    departed_boolean = row['departed']
    arrived_boolean = row['arrived']

    scheduled_time = row['scheduled']
    estimated_time = row['estimated']
    time_diff = row['diffMin']
    train_departure_schedule = row['train_departure_schedule']

    #get to index
    cur.execute(query_location, (location_i,))
    row = cur.fetchone()
    location_idx = row[0]

    # Check if next_location_i is not None before proceeding
    if pd.isna(next_location_i):
        if(location_i == to_i):
            # If the current location is the same as the destination
            next_location_idx = location_idx
        #if stop and end of route are not the same then move forward
        else:
            continue
    else:
        cur.execute(query_location, (next_location_i.upper(),))
        row = cur.fetchone()
        next_location_idx = row[0]

    #get to index
    cur.execute(query_location, (to_i,))
    row = cur.fetchone()
    to_idx = row[0]

    #get from index
    cur.execute(query_location, (from_i,))
    row = cur.fetchone()
    from_idx = row[0]

    #get route
    cur.execute(query_route, (from_idx, to_idx))
    row = cur.fetchone()
    route_idx = row[0]

    #get train
    cur.execute(query_train, (train_num, route_idx))
    row = cur.fetchone()
    train_idx = row[0]

    #get stop
    cur.execute(query_stop, (train_idx, location_idx, next_location_idx))
    row = cur.fetchone()
    stop_idx = row[0]

    #If train has arrived and departed then save data
    if departed_boolean and arrived_boolean:
        print("write to database")
        data_tuple = (stop_idx, scheduled_time.strftime("%Y-%m-%d %H:%M:%S"), estimated_time.strftime("%Y-%m-%d %H:%M:%S"), time_diff, train_departure_schedule.strftime("%Y-%m-%d"))
        return_v = cur.execute(sqlite_insert_with_param, data_tuple)
        #print(scheduled_time)
        #print(scheduled_time.strftime("%Y-%m-%d"))
con.commit()
#cur.close()

write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to database
write to d

In [72]:
con.commit()